In [265]:
import requests, json

In [266]:
from bs4 import BeautifulSoup

In [267]:
# 23254444, EX5GHDZV
payload = "{\n    \"password\": \"Pp4581831265\",\n    \"username\": \"p.promto@gmail.com\"\n}"

In [268]:
headers = {
    'Content-Type': "application/json",
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "9b2fa27c-4052-477a-94f0-9f2482765e41,392e6b9e-f3ad-49db-bf29-2b4e03cfb78d",
    'Host': "api.setsmart.com",
    'Accept-Encoding': "gzip, deflate",
    'Content-Length': "66",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
}

In [269]:
# Login
with requests.Session() as s:
    url = 'https://api.setsmart.com/api/user/login'
    r = s.post(url, data=payload, headers=headers)
    
    ACCESS_TOKEN = r.json()['access_token']
    
    r_create_session = s.get('https://www.setsmart.com/ism/api/createusersession.api', headers={'Authorization': 'Bearer ' + ACCESS_TOKEN})
    
    JSESSIONID = s.cookies.get_dict()['JSESSIONID']
    print('JSESSIONID', JSESSIONID)

JSESSIONID 88DFEDA75E2D040D27D9CA7D632DED0D.localhost


In [278]:
# Get Page
query_headers = {
    'Cookie': "JSESSIONID=" + JSESSIONID + ";",
    'User-Agent': "PostmanRuntime/7.15.2",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "2becce72-e79c-44a5-b7c7-e6627123df59,85ebc553-9da8-425f-8ee5-954172b9fe3b",
    'Host': "www.setsmart.com",
    'Accept-Encoding': "gzip, deflate",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
}

print(query_headers)
querystring = { "locale":"en_US", "symbol":"AAV01C1408A" }
url = 'https://www.setsmart.com/ism/companyprofile.html'
r = s.get(url, headers=query_headers, params=querystring)
print(r)
# html_doc = r.content

{'Cookie': 'JSESSIONID=88DFEDA75E2D040D27D9CA7D632DED0D.localhostl;', 'User-Agent': 'PostmanRuntime/7.15.2', 'Accept': '*/*', 'Cache-Control': 'no-cache', 'Postman-Token': '2becce72-e79c-44a5-b7c7-e6627123df59,85ebc553-9da8-425f-8ee5-954172b9fe3b', 'Host': 'www.setsmart.com', 'Accept-Encoding': 'gzip, deflate', 'Connection': 'keep-alive', 'cache-control': 'no-cache'}
<Response [200]>


In [271]:
# soup = BeautifulSoup(html_doc, 'html.parser')
# data_url = soup.iframe.get('src')

In [272]:
# #Get Data
# url = 'https://www.setsmart.com' + str(data_url);
# r = s.get(url)
# soup = BeautifulSoup(r.content, 'html.parser')

# str_data = soup.pre.string.replace(" ", "")

In [273]:
# # find data
# col_name = ['DWValue(Baht)', 'Exerciseratio(DW:Underlyingasset)', 'Exerciseprice(baht)']

# for name in col_name:
#     index_start = str_data.find(name)
#     index_end = str_data.find('\n', index_start)
#     data = str_data[index_start + len(name) + 1:index_end]
#     print(name, data)

In [274]:
# urlLine = 'https://notify-api.line.me/api/notify'
# token = 'nVCDZg1I4UiztlhcZe2C14MhUCijNUX4421KcjUmBeZ'
# headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + token}

# s.post(urlLine, headers=headers , data = {'message': 'Stringing Scraping'})

In [275]:
import csv
from IPython.display import clear_output

In [276]:
codes = []
with open('./raw_data/code.csv', mode='r', encoding='utf-8-sig') as code_file:
    csv_reader = csv.reader(code_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        codes.append([str(row[0])])
        line_count += 1
code_file.close()

In [277]:
start_row = 0
end_row = 1

with open('./process_data/data-' + str(start_row+1) + '-' + str(end_row) + '.csv', 'w+') as csv_file:
    writer = csv.writer(csv_file)
    
    for code in codes[start_row:end_row]:
        
        clear_output(wait=True)
        
        com_code = code[0]
        
        print(f'Prepare com code {com_code}')
        
        try:
            # Get Page
            querystring = { "locale": "en_US", "symbol": com_code }
            url = 'https://www.setsmart.com/ism/companyprofile.html'
            r = requests.request("GET",url, headers=query_headers, params=querystring)
            html_doc = r.content
        except requests.exceptions.RequestException as e:  # This is the correct syntax
            s.post(urlLine, headers=headers , data = {'message': 'network error when scraping ' + url})
            sys.exit(1)
        
        soup = BeautifulSoup(html_doc, 'html.parser')
        
        print(r, com_code)
        
        if soup.iframe == None:
            writer.writerow(code)
            continue
        
        print('data Url : ' , soup.find_all('iframe')[1])
        
        data_url = soup.find_all('iframe')[1].get('src')
        
        #find ...
        data_no_issued_shares = soup.find('td', text="No. of Issued Shares").find_next_sibling('td').string
        print('data_no_issued_shares', data_no_issued_shares)
        
        try:
            #Get Data
            str_data = ""
            url = 'https://www.setsmart.com' + str(data_url);
            r = s.get(url, headers=query_headers)
            soup = BeautifulSoup(r.content, 'html.parser')
            str_data = soup.pre.string.replace(" ", "").lower()
            print(str_data)
        except requests.exceptions.RequestException as e:  # This is the correct syntax
            s.post(urlLine, headers=headers , data = {'message': 'network error when scraping : ' + url})
            sys.exit(1)
        
        # find data
        col_name = [
            'dwvalue(baht):', #0
            'exerciseratio(dw:underlyingasset):', #1
            'exerciseratio(calculate):', #2
            'exerciseprice(baht):', #3
            'tradingdate:', #4
            'lasttradingdate:', #5
            'maturitydate:', #6
            'lastexercisedate:', #7
        ]
        
        get_data = {}
        for name in col_name:
            index_start = str_data.find(name, 0)
            index_end = str_data.find('\n', index_start)
            
            if index_start == -1:
                if name == 'dwvalue(baht):':
                    index_start = str_data.find('dwvalue(bath):', 0)
                    index_end = str_data.find('\n', index_start)
                    if index_start == -1:
                        get_data[name] = "!!!! Fail !!!! - Not found " + str(name)
                        continue
                else:
                    get_data[name] = ""
                    continue

            data = str_data[index_start + len(name):index_end]
            get_data[name] = data

        code = code + list(map(lambda name: get_data[name], col_name))
        
        code = code + [data_no_issued_shares.replace('\n', '').replace('\r', '').replace('\xa0', '').replace(' ', '').replace('Shares', '')]
        
        print(f'Data com code {code}')
        
        writer.writerow(code)
csv_file.close()

Prepare com code AAV01C1408A
<Response [200]> AAV01C1408A
data Url :  <iframe height="150" src="/ism/securitydetailfile.html?filename=dat%2fsecurity%2f00020416.eng" width="100%"> </iframe>
data_no_issued_shares 59,870,600
     Shares
    

listing

listing:derivativewarrant
dwissuer:bualuangsecuritiespubliccompany
limited(bls)
dwname:aav01c1408a
secondarymarket:set
tradingdate:26-feb-2014
numberofderivativewarrants(unit::60,000,000
dws)
underlyingasset:aav
dwvalue(baht):60,000,000.00
exerciseratio(dw:underlyingasset):2:1
exerciseprice(baht):3.30
typeofdw:callwarrant
exercisetype:europeanstyle
lasttradingdate:29-aug-2014
maturitydate:04-sep-2014
lastexercisedate:04-sep-2014
marketmaker:bualuangsecuritiespubliccompany
limited(bls)
settlementcondition(s):cashsettlement
remark:
pleaseseedescription,termsandconditionsofdwabove-mentionedthroughpdf
filedisseminatedonthesetmarketanalysisandreportingtool(setsmart).
______________________________________________________________________


Data co